<a href="https://colab.research.google.com/github/Akasxh/Stock_Analysis/blob/main/edgar_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install beautifulsoup4

!pip install sec-edgar-downloader

!pip install openpyxl

!pip install arelle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for arelle: filename=arelle-2.2-py3-none-any.whl size=2060281 sha256=812fa9bcc07926c599ef7b2a708908055cde66256231d2fdfd734a6c6cf4eee4
  Stored in directory: /root/.cache/pip/wheels/fc/68/39/f091b0572329199f0ecce0d6c49afe79c3c9f10c5a31ad56b0
Successfully built arelle


In [13]:
from sec_edgar_downloader import Downloader

import os
import glob
import re
from bs4 import BeautifulSoup
import pandas as pd
from textblob import TextBlob
from datetime import datetime


In [14]:
# Initialize the downloader
dl = Downloader("YOUR_COMPANY_NAME", "YOUR_EMAIL_ADDRESS","/content")

In [6]:
def extract_text_from_html(file_path):
    # Ensure the file exists
    if not os.path.exists(file_path):
        return "File does not exist"

    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract text from the parsed HTML
    text = soup.get_text(separator='\n', strip=True)

    return text

In [23]:
def truncate_and_overwrite(file_path, target_line='XBRL INSTANCE DOCUMENT'):
    """
    Truncates the file content after a specific target line and overwrites the original file.

    :param file_path: Path to the file to be modified.
    :param target_line: The line after which all text should be discarded.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as infile:
            lines = infile.readlines()

        # Determine the point to truncate
        truncation_point = None
        for i, line in enumerate(lines):
            if target_line in line:
                truncation_point = i + 1  # Include the target line and stop after
                break

        # If the target line is found, truncate the lines
        if truncation_point is not None:
            lines = lines[:truncation_point]
            # Overwrite the original file with truncated content
            with open(file_path, 'w', encoding='utf-8') as outfile:
                outfile.writelines(lines)
        else:
            print(f"Target line '{target_line}' not found in file.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [26]:
def merge(company_ticker):
    dl.get("10-K", company_ticker.upper(), after="1995-01-01", before="2023-01-01", download_details=False)
    base_directory = f'/content/sec-edgar-filings/{company_ticker}/10-K'
    file_paths = glob.glob(os.path.join(base_directory, '**', '*.txt'), recursive=True)

    for file_path in file_paths:
            truncate_and_overwrite(file_path)

    #output_filename = f'/content/merged_output_{company_ticker.upper()}.txt'
    '''with open(output_filename, 'w') as output_file:
        for file_path in file_paths:
            truncate_and_overwrite(file_path)
            #extracted_data = extract_text_from_html(file_path)
            #output_file.write(extracted_data + '\n')'''


In [17]:
def visualise(company_ticker):
    merge(company_ticker)

In [18]:
def main():
    company_ticker = input("Enter the company ticker: ")
    visualise(company_ticker)

In [19]:
main()

Enter the company ticker: NFLX


In [24]:
truncate_and_overwrite("/content/sec-edgar-filings/NFLX/10-K/0001065280-15-000006/full-submission.txt")